In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import flax.linen as nn
import matplotlib.pyplot as plt
import os
from tifffile import imread, imsave
import torch
import torch.utils.data as dt

In [3]:
os.chdir('/mnt/c/Users/ankit/Desktop/Msc AIML/msc-project/GAP-JAX')
# os.chdir(r'C:\Users\ankit\Desktop\Msc AIML\msc-project\GAP-JAX')
from gap_jax.BinomDataset_JAX import BinomDataset
from  gap_jax.GAP_JAX_Trainer import Trainer
from gap_jax.GAP_UNET_ResBlock_JAX import UN

In [3]:
data = imread('/mnt/c/Users/ankit/Desktop/Msc AIML/Data/convallaria/trainingDataGT.tif')
# data = imread(r'C:\Users\ankit\Desktop\Msc AIML\Data\convallaria\trainingDataGT.tif')


In [4]:
print(data.shape)

(1536, 512, 512)


In [11]:
def psnrToString(inp):
    if inp < 0:
        return 'm'+str(-inp)
    else:
        return str(inp)

minpsnr = -40
maxpsnr = -5

name = psnrToString(minpsnr)+"to"+psnrToString(maxpsnr)+"-256x256-jax-v1"
CHECKPOINT_PATH = '/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/jax/wsl/'
# CHECKPOINT_PATH  = "C:\\Users\\ankit\\Desktop\\Msc AIML\\Data\\checkpoints\\models\\jax\\windows\\"
CHECKPOINT_PATH, name

('/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/jax/wsl/',
 'm40tom5-256x256-jax-v1')

In [6]:
maxepochs = 20 # train longer for better results

# Create pytorch dataset
dataset = BinomDataset(data = data[:round(data.shape[0]*0.9)], windowSize = 256, minPSNR = minpsnr, maxPSNR = maxpsnr)
val_dataset = BinomDataset(data = data[round(data.shape[0]*0.9):], windowSize = 256, minPSNR = minpsnr, maxPSNR = maxpsnr)

# Create pytorch dataloader for training and validation sets
train_loader = dt.DataLoader(dataset, batch_size=1, shuffle=True, drop_last=True, pin_memory=False, num_workers=12, collate_fn= lambda batch: np.array(batch)) ## Changing the batch size from 32 to 16 to fit inside the gpu
val_loader = dt.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=False,  pin_memory=False, num_workers=12, collate_fn= lambda batch: np.array(batch))

# img =  next(iter(val_loader))

# print(img.shape)

In [4]:
model = UN(channels = 1, levels=10, depth=3, start_filts=28, up_mode = 'transpose')

In [8]:
trainer = Trainer(root_dir= os.path.join(CHECKPOINT_PATH, name),
                  gradient_clip_val=0.5,
                  epochs= maxepochs,
                  dataloader= train_loader,
                  model = model)

GPU detected with 1 Device(s).


In [9]:
trainer.train_model(train_loader= train_loader,
                    val_loader= val_loader)

  0%|          | 0/20 [00:00<?, ?it/s]

/home/ankith/anaconda3/envs/rsenv/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training:   0%|          | 0/172 [00:00<?, ?it/s]

2024-07-19 22:09:26.681233: W external/xla/xla/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 29.24GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
/home/ankith/anaconda3/envs/rsenv/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/ankith/anaconda3/envs/rsenv/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


ValueError: NOT_FOUND: Error writing local file "/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/jax/m40tom5-256x256-jax-v1/checkpoint_1.orbax-checkpoint-tmp-0/_METADATA":  [OS error 2: No such file or directory] [source locations='tensorstore/internal/os/file_util_posix.cc:250\ntensorstore/kvstore/file/file_key_value_store.cc:695\ntensorstore/kvstore/transaction.cc:122']

In [5]:
img = jnp.ones((1, 256, 256, 1))

In [6]:
params = model.init(jax.random.key(68), img)

In [7]:
params

{'params': {'down_convs_0': {'conv1': {'kernel': Array([[[[ 0.04835897, -0.01130721,  0.13503538, ...,  0.06476042,
              -0.00980399,  0.13382831],
             [-0.11486074, -0.06955706,  0.06231094, ...,  0.00211302,
              -0.06623932, -0.12104253],
             [-0.08883706, -0.04109567,  0.06634794, ..., -0.11131057,
              -0.05823373, -0.10369347],
             ...,
             [ 0.1004604 , -0.05761833,  0.10426241, ..., -0.07090598,
              -0.05718922, -0.0832227 ],
             [ 0.15443651, -0.02431712, -0.054013  , ...,  0.00984212,
              -0.04689721, -0.06521375],
             [ 0.07888078,  0.0981305 , -0.10198117, ...,  0.06014301,
              -0.15116563, -0.07633726]],
    
            [[-0.13767436, -0.13606043,  0.06190977, ...,  0.00630517,
               0.0546028 ,  0.08283786],
             [ 0.05103631, -0.03247613, -0.05663961, ..., -0.02367956,
              -0.08064031, -0.07050055],
             [ 0.13343303,  0.05339

In [8]:
out1 = model.apply(params, img)

In [9]:
np.unique(out1 == out1)

array([ True])

In [15]:
# CHECKPOINT_PATH = '/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/jax/del/'
# CHECKPOINT_PATH, name
windows_path = CHECKPOINT_PATH
wsl_path = 

In [14]:
from flax.training import checkpoints
checkpoints.save_checkpoint(ckpt_dir="/home/anki",
                                    target=params,
                                    step = 0,
                                    overwrite=True)

ValueError: NOT_FOUND: Error writing local file "/mnt/c/Users/ankit/Desktop/checkpoint_0.orbax-checkpoint-tmp-3/_METADATA":  [OS error 2: No such file or directory] [source locations='tensorstore/internal/os/file_util_posix.cc:250\ntensorstore/kvstore/file/file_key_value_store.cc:695\ntensorstore/kvstore/transaction.cc:122']